<center>

**CURVAS ELIPTICAS**

</center>

<p align="center">
    <img src="https://logowik.com/content/uploads/images/escudo-de-la-universidad-nacional-de-colombia-20163327.logowik.com.webp" width="400">
</p>

<center>

# **⛎Curvas Elípticas♉**

<p align="center">
    <img src="https://www.ecured.cu/images/7/71/ECDSA.jpg" width="600">
</p>

<div align="justify">

La criptografía de curvas elípticas se basa en el grupo abeliano de puntos sobre una curva elíptica definida por la ecuación
$$
y^2 = x^3 + a\,x + b \pmod{p}
$$
donde $p$ es un número primo, y los parámetros $a,b\in\mathbb{F}_p$ satisfacen que el discriminante $\Delta = -16(4a^3 + 27b^2)\neq 0$.  
- **Punto en el infinito** $\mathcal{O}$ actúa como elemento neutro del grupo.  
- **Suma de puntos**: dados $P=(x_1,y_1)$ y $Q=(x_2,y_2)$, su suma $R=P+Q=(x_3,y_3)$ se define mediante
  $$
  \lambda =
  \begin{cases}
    \displaystyle\frac{3x_1^2 + a}{2y_1}\!\!\mod p,& P=Q,\,y_1\neq 0,\\[6pt]
    \displaystyle\frac{y_2 - y_1}{x_2 - x_1}\!\!\mod p,& P\neq Q,\,x_2\neq x_1,
  \end{cases}
  $$
  $$
  x_3 = \lambda^2 - x_1 - x_2 \pmod{p},\quad
  y_3 = \lambda\,(x_1 - x_3) - y_1 \pmod{p}.
  $$  
- **Multiplicación escalar** $kP = P + P + \cdots + P$ ($k$ veces) se implementa eficientemente con el método “doblar y sumar”.

---
**Generación de Claves en ECDSA**

1. **Selección de parámetros**:  
   - Curva: $(p,a,b)$  
   - Punto generador de orden $n$: $G\in E(\mathbb{F}_p)$ tal que $nG=\mathcal{O}$.  
   - Factor de cofáctor $h = \lvert E(\mathbb{F}_p)\rvert / n$.  

2. **Clave privada** $d$: número entero aleatorio en $[1,\,n-1]$.  
3. **Clave pública** $Q$: punto de la curva
   $$
   Q = d\,G.
   $$

---
**Firma Digital con ECDSA**

Para firmar un mensaje $M$:

1. Calcular el hash criptográfico (por ejemplo SHA‑256) y reducirlo:
   $$
   z = \bigl\lfloor \mathrm{Hash}(M)\bigr\rfloor_{\,\bmod\,n}.
   $$
2. Elegir un nonce aleatorio $k\in[1,n-1]$ y calcular
   $$
   R = k\,G = (x_R, y_R),\qquad r = x_R \bmod n.
   $$
   Repetir si $r=0$.
3. Calcular el inverso modular $k^{-1}\bmod n$.
4. Calcular
   $$
   s = k^{-1}\,(z + r\,d)\bmod n.
   $$
   Repetir si $s=0$.
5. La firma es el par $(r,s)$.

> **Importante:** $k$ debe ser único y aleatorio para cada firma. La reutilización de $k$ compromete la clave privada.

---
**Verificación de la Firma**

Dado el mensaje $M$, la clave pública $Q$ y la firma $(r,s)$:

1. Verificar que $r,s\in[1,n-1]$.  
2. Calcular $z = \lfloor\mathrm{Hash}(M)\rfloor_{\bmod n}$ y el inverso $w = s^{-1}\bmod n$.  
3. Calcular
   $$
   u_1 = z\,w \bmod n,\quad u_2 = r\,w \bmod n.
   $$
4. Obtener los puntos
   $$
   P_1 = u_1\,G,\quad P_2 = u_2\,Q,\quad X = P_1 + P_2 = (x_X, y_X).
   $$
5. La firma es **válida** si $X\neq\mathcal{O}$ y
   $$
   x_X \bmod n = r.
   $$
---
**Seguridad y Buenas Prácticas**

- **Dificultad del problema**: La seguridad reside en la dificultad del _Discrete Logarithm Problem_ en curvas elípticas (ECDLP).  
- **Parámetros recomendados**: Curvas estandarizadas (p. ej. secp256k1, prime256v1) con $p\approx 2^{256}$.  
- **Generación de entropía**: Clave privada $d$ y nonce $k$ deben provenir de un CSPRNG fiable.  
- **Protección contra ataques de canal lateral**: Implementaciones seguras que eviten fugas de información de $k$ o $d$.  
- **Validación de puntos**: Antes de usar $Q$ o $R$, comprobar que pertenecen a la curva y que $n\,Q=\mathcal{O}$.



</div>

<center>

**📥Importaciones📦**

In [4]:

!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [5]:

from IPython.display import display, HTML
import ipywidgets as widgets
from hashlib import sha256
from random import randint
from sympy import mod_inverse

**👨‍💻Implementación👩‍💻**

In [6]:

p = 17
a = 2
b = 2
G = (5, 1)
n = 19

# === Funciones ECC ===
def ecc_add(P, Q):
    if P is None: return Q
    if Q is None: return P
    if P == Q:
        if P[1] == 0: return None
        m = ((3 * P[0]**2 + a) * mod_inverse(2 * P[1], p)) % p
    else:
        if P[0] == Q[0]: return None
        m = ((Q[1] - P[1]) * mod_inverse(Q[0] - P[0], p)) % p
    x_r = (m**2 - P[0] - Q[0]) % p
    y_r = (m * (P[0] - x_r) - P[1]) % p
    return (x_r, y_r)

def ecc_mul(k, P):
    R = None
    for _ in range(k):
        R = ecc_add(R, P)
    return R

# === Firma y Verificación ECDSA ===
def ecdsa_sign(message, d):
    z = int(sha256(message.encode()).hexdigest(), 16) % n
    while True:
        k = randint(1, n - 1)
        R = ecc_mul(k, G)
        if R is None: continue
        r = R[0] % n
        if r == 0: continue
        try:
            k_inv = mod_inverse(k, n)
        except:
            continue
        s = (k_inv * (z + r * d)) % n
        if s == 0: continue
        return (r, s)

def ecdsa_verify(message, Q, signature):
    r, s = signature
    z = int(sha256(message.encode()).hexdigest(), 16) % n
    try:
        w = mod_inverse(s, n)
    except:
        return False
    u1 = (z * w) % n
    u2 = (r * w) % n
    P1 = ecc_mul(u1, G)
    P2 = ecc_mul(u2, Q)
    X = ecc_add(P1, P2)
    return X is not None and (X[0] % n) == r

# === Widgets UI ===
style = {'description_width': '150px'}
layout_input = widgets.Layout(width='400px', height='40px')
layout_area = widgets.Layout(width='600px', height='150px')

# Persona A - Firmante
privA = widgets.IntText(value=3, description="🔑 Clave privada A:", style=style, layout=layout_input)
msgA = widgets.Textarea(value="Hola, este mensaje será firmado.", description="📝 Mensaje:", style=style, layout=layout_area)
sign_button = widgets.Button(description="✍️ Firmar y Enviar", button_style="success", layout=widgets.Layout(width='250px', height='45px'))

# Persona B - Verificador
pubB = widgets.Text(value='', description="🔓 Clave pública A:", style=style, layout=layout_input)
msgB = widgets.Textarea(value='', description="📨 Mensaje recibido:", style=style, layout=layout_area)
sigR = widgets.IntText(value=0, description="🔐 Firma r:", style=style, layout=layout_input)
sigS = widgets.IntText(value=0, description="🔐 Firma s:", style=style, layout=layout_input)
verify_button = widgets.Button(description="🔎 Verificar Firma", button_style="info", layout=widgets.Layout(width='250px', height='45px'))

# Salida de mensajes
output = widgets.Output(layout={'border': '1px solid gray', 'padding': '10px'})
signature_shared = {}

# === Funciones de evento ===
def on_sign_clicked(b):
    output.clear_output()
    d = privA.value
    msg = msgA.value
    Q = ecc_mul(d, G)
    r, s = ecdsa_sign(msg, d)
    signature_shared['Q'] = Q
    signature_shared['msg'] = msg
    signature_shared['sig'] = (r, s)

    # "Envío" de datos a B
    pubB.value = str(Q)
    msgB.value = msg
    sigR.value = r
    sigS.value = s

    with output:
        display(HTML(f"""
        <h4 style="color:green;">✅ Firma creada y enviada a B</h4>
        <p><b>Clave pública A:</b> {Q}</p>
        <p><b>Firma (r, s):</b> ({r}, {s})</p>
        """))

def on_verify_clicked(b):
    output.clear_output()
    try:
        Q_str = pubB.value.replace('(', '').replace(')', '')
        Qx, Qy = map(int, Q_str.split(','))
        Q = (Qx, Qy)
        r = sigR.value
        s = sigS.value
        msg = msgB.value
        valid = ecdsa_verify(msg, Q, (r, s))
        with output:
            if valid:
                display(HTML("<p style='color:green; font-size:16px;'>✅ Firma válida. Mensaje verificado con éxito.</p>"))
            else:
                display(HTML("<p style='color:red; font-size:16px;'>❌ Firma inválida. El mensaje puede haber sido alterado.</p>"))
    except:
        with output:
            display(HTML("<p style='color:red;'>⚠️ Error procesando datos. Verifica el formato de clave pública y firma.</p>"))

# Asignar eventos
sign_button.on_click(on_sign_clicked)
verify_button.on_click(on_verify_clicked)

# === Mostrar la app ===
display(HTML("<h2 style='color:#00cc99;'> Firma y Verificación con ECDSA</h2>"))
display(HTML("<h3>👤 Persona A: Firmante</h3>"))
display(widgets.VBox([privA, msgA, sign_button]))
display(HTML("<hr style='margin:20px 0;'>"))
display(HTML("<h3>👤 Persona B: Verificador</h3>"))
display(widgets.VBox([pubB, msgB, widgets.HBox([sigR, sigS]), verify_button]))
display(widgets.HTML("<br>"))
display(output)


HTML(value='<br>')

Output(layout=Layout(border='1px solid gray', padding='10px'))